In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as sf

In [2]:
spark = SparkSession \
        .builder \
        .master("spark://spark-master:7077") \
        .config(
        "spark.jars",
        "/opt/bitnami/spark/jars/gcs-connector-hadoop3-latest.jar,"
        "/opt/bitnami/spark/jars/spark-bigquery-with-dependencies_2.12-0.42.4.jar"
        ) \
        .appName('gcs-bq-pyspark') \
        .getOrCreate()

25/08/13 08:35:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.sparkContext.setLogLevel("WARN")

spark._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
spark._jsc.hadoopConfiguration().set('fs.gs.auth.service.account.enable', 'true')
spark._jsc.hadoopConfiguration().set('google.cloud.auth.service.account.json.keyfile', "/opt/keys/credentials.json")

In [4]:
# Read from CSV files
bucket_name = "data_expo_bucket"
files = [f"gs://{bucket_name}/{year}_data.csv" for year in range(2000, 2009)]
files

['gs://data_expo_bucket/2000_data.csv',
 'gs://data_expo_bucket/2001_data.csv',
 'gs://data_expo_bucket/2002_data.csv',
 'gs://data_expo_bucket/2003_data.csv',
 'gs://data_expo_bucket/2004_data.csv',
 'gs://data_expo_bucket/2005_data.csv',
 'gs://data_expo_bucket/2006_data.csv',
 'gs://data_expo_bucket/2007_data.csv',
 'gs://data_expo_bucket/2008_data.csv']

In [5]:
df = spark.read.option("header", "true").csv(files)

In [8]:
# Check DataFrame's schema
df.schema

StructType([StructField('Year', StringType(), True), StructField('Month', StringType(), True), StructField('DayofMonth', StringType(), True), StructField('DayOfWeek', StringType(), True), StructField('DepTime', StringType(), True), StructField('CRSDepTime', StringType(), True), StructField('ArrTime', StringType(), True), StructField('CRSArrTime', StringType(), True), StructField('UniqueCarrier', StringType(), True), StructField('FlightNum', StringType(), True), StructField('TailNum', StringType(), True), StructField('ActualElapsedTime', StringType(), True), StructField('CRSElapsedTime', StringType(), True), StructField('AirTime', StringType(), True), StructField('ArrDelay', StringType(), True), StructField('DepDelay', StringType(), True), StructField('Origin', StringType(), True), StructField('Dest', StringType(), True), StructField('Distance', StringType(), True), StructField('TaxiIn', StringType(), True), StructField('TaxiOut', StringType(), True), StructField('Cancelled', StringType

In [9]:
# Take a look at some records
df.take(5)

[Row(Year='2000', Month='1', DayofMonth='28', DayOfWeek='5', DepTime='1647', CRSDepTime='1647', ArrTime='1906', CRSArrTime='1859', UniqueCarrier='HP', FlightNum='154', TailNum='N808AW', ActualElapsedTime='259', CRSElapsedTime='252', AirTime='233', ArrDelay='7', DepDelay='0', Origin='ATL', Dest='PHX', Distance='1587', TaxiIn='15', TaxiOut='11', Cancelled='0', CancellationCode='NA', Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA'),
 Row(Year='2000', Month='1', DayofMonth='29', DayOfWeek='6', DepTime='1648', CRSDepTime='1647', ArrTime='1939', CRSArrTime='1859', UniqueCarrier='HP', FlightNum='154', TailNum='N653AW', ActualElapsedTime='291', CRSElapsedTime='252', AirTime='239', ArrDelay='40', DepDelay='1', Origin='ATL', Dest='PHX', Distance='1587', TaxiIn='5', TaxiOut='47', Cancelled='0', CancellationCode='NA', Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA'),
 Row(Yea

In [10]:
# Drop NULL columns
cols_to_drop = ['CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
df = df.drop(*cols_to_drop)

In [11]:
# Check DataFrame's schema after dropping columns
df.schema

StructType([StructField('Year', StringType(), True), StructField('Month', StringType(), True), StructField('DayofMonth', StringType(), True), StructField('DayOfWeek', StringType(), True), StructField('DepTime', StringType(), True), StructField('CRSDepTime', StringType(), True), StructField('ArrTime', StringType(), True), StructField('CRSArrTime', StringType(), True), StructField('UniqueCarrier', StringType(), True), StructField('FlightNum', StringType(), True), StructField('TailNum', StringType(), True), StructField('ActualElapsedTime', StringType(), True), StructField('CRSElapsedTime', StringType(), True), StructField('AirTime', StringType(), True), StructField('ArrDelay', StringType(), True), StructField('DepDelay', StringType(), True), StructField('Origin', StringType(), True), StructField('Dest', StringType(), True), StructField('Distance', StringType(), True), StructField('TaxiIn', StringType(), True), StructField('TaxiOut', StringType(), True), StructField('Cancelled', StringType

In [12]:
# Repartition for safe parallelism
df = df.repartition(40)

In [ ]:
# Write to Parquet in GCS
df.write.mode("overwrite").parquet(f"gs://{bucket_name}/all_years_parquet")

[Stage 3:====>                                                     (3 + 1) / 41]